In [11]:
import pandas as pd
import numpy as np
import os
from sklearn import metrics
from glob import glob
from datetime import datetime
import time
import json

# Sentiment analysis, Offensive language and emotion recognition



In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, AutoConfig
import torch
from scipy.special import softmax
import csv
import urllib.request

## Sentiment analysis

In [ ]:
# labels retrival
labels_sentiment=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels_sentiment = [row[1] for row in csvreader if len(row) > 1]

print(labels_sentiment)

['negative', 'neutral', 'positive']


In [ ]:
# Classification throught sentiment analysis in 3 labels: 'Negative', 'Neutral' or 'Positive'
sentiment_analysis_model = "cardiffnlp/twitter-roberta-base-sentiment-latest"
model_sentiment = AutoModelForSequenceClassification.from_pretrained(sentiment_analysis_model)
tokenizer_sentiment = AutoTokenizer.from_pretrained(sentiment_analysis_model)

def sentiment_analysis(text, model, tokenizer, lables):
  labels = ['Negative', 'Neutral', 'Positive']
  # Padding true for allow the model to have the same dimension for all the data that we pass. it will automatically use mask value as 0 for text with padding (so will not be considered).
  # 'pt' means a pytorch output
  encoded_tweet = tokenizer(text, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
  # It doesn't update the gradients and so the computational time decrease
  with torch.no_grad():
    output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  max_index = scores.argmax()
  max_score = scores[max_index]
  max_label = labels_sentiment[max_index]

  return max_label

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\Castagna\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
 

In [ ]:
sentiment_analysis('i am very happy to be here', model_sentiment, tokenizer_sentiment, labels_sentiment)

'positive'

## Emotion recognition

In [ ]:
# labels retrival
labels_emotion=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels_emotion = [row[1] for row in csvreader if len(row) > 1]

print(labels_emotion)

['anger', 'joy', 'optimism', 'sadness']


In [ ]:
task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
model_emotion = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer_emotion = AutoTokenizer.from_pretrained(MODEL)

def emotion_recognition(text, model, tokenizer, labels):
  # Padding true for allow the model to have the same dimension for all the data that we pass. it will automatically use mask value as 0 for text with padding (so will not be considered).
  # 'pt' means a pytorch output
  # truncated means that, based on how the model has been trained, if a sentence is longer than the maximum handable from the model, the sentence will be truncated.
  encoded_tweet = tokenizer(text, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

  # The output is a unnormalized score that needs to be pass to a softmax function for asses the exact percentage
  with torch.no_grad():
    output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  max_index = scores.argmax()
  max_score = scores[max_index]
  max_label = labels[max_index]

  return max_label

## Offensive language

In [ ]:
# label retrival
labels_offensive=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels_offensive = [row[1] for row in csvreader if len(row) > 1]
print(labels_offensive)

['not-offensive', 'offensive']


In [ ]:
task='offensive'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
model_offensive = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer_offensive = AutoTokenizer.from_pretrained(MODEL)

def offensive_language(text, model, tokenizer, labels):
  # Padding true for allow the model to have the same dimension for all the data that we pass. it will automatically use mask value as 0 for text with padding (so will not be considered).
  # 'pt' means a pytorch output
  encoded_tweet = tokenizer(text, max_length=512, padding='max_length', truncation=True, return_tensors='pt')

  # The output is a unnormalized score that needs to be pass to a softmax function for asses the exact percentage
  with torch.no_grad():
    output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  max_index = scores.argmax()
  max_score = scores[max_index]
  max_label = labels[max_index]

  return max_label

# Posts 

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn import metrics
from glob import glob
from datetime import datetime
import time
import json

file_zip = 'insert path'       #"C:\\Users\\aless\\Desktop\\reddit_distr\\posts"

os.chdir(file_zip)

print("Files estratti:")
print(os.listdir())
rewards=sorted(os.listdir())
rewards=rewards[9:]
rewards

Files estratti:
['r_ethtrader_comments_2016.jsonl', 'r_ethtrader_comments_2017.jsonl', 'r_ethtrader_comments_2018.jsonl', 'r_ethtrader_comments_2018_bkp.jsonl', 'r_ethtrader_comments_2019.jsonl', 'r_ethtrader_comments_2020.jsonl', 'r_ethtrader_comments_2021.jsonl', 'r_ethtrader_comments_2022.jsonl', 'r_ethtrader_comments_2023.jsonl', 'r_ethtrader_posts_2016.jsonl', 'r_ethtrader_posts_2017.jsonl', 'r_ethtrader_posts_2018.jsonl', 'r_ethtrader_posts_2019.jsonl', 'r_ethtrader_posts_2020.jsonl', 'r_ethtrader_posts_2021.jsonl', 'r_ethtrader_posts_2022.jsonl', 'r_ethtrader_posts_2023.jsonl']


['r_ethtrader_posts_2016.jsonl',
 'r_ethtrader_posts_2017.jsonl',
 'r_ethtrader_posts_2018.jsonl',
 'r_ethtrader_posts_2019.jsonl',
 'r_ethtrader_posts_2020.jsonl',
 'r_ethtrader_posts_2021.jsonl',
 'r_ethtrader_posts_2022.jsonl',
 'r_ethtrader_posts_2023.jsonl']

In [4]:
selected_file = rewards
all_data=pd.DataFrame()
for i in selected_file:
  print(i)
  author=[]
  text=[]
  up_down=[]
  year=[]
  retrieved_on=[]
  retrieved_utc=[]
  created_utc=[]
  link_flair_text=[]
  num_comments=[]
  with open(i, 'r', encoding='UTF-8') as f:
      for line in f:
        line=json.loads(line)
        author.append(line['author'])
        all_text=line['title'] + line['selftext']
        text.append(all_text.replace('\n', ' '))
        up_down.append(line['ups'])
        retrieved_on.append(line.get('retrieved_on', np.nan))
        retrieved_utc.append(line.get('retrieved_utc', np.nan))
        created_utc.append(line.get('created_utc', np.nan))
        link_flair_text.append(line.get('link_flair_text', np.nan))
        num_comments.append(line.get('num_comments', np.nan))
        if '2016' in i:
          year.append('2016')
        elif '2017' in i:
          year.append('2017')
        elif '2018' in i:
          year.append('2018')
        elif '2019' in i:
          year.append('2019')
        elif '2020' in i:
          year.append('2020')
        elif '2021' in i:
          year.append('2021')
        elif '2022' in i:
          year.append('2022')
        elif '2023' in i:
          year.append('2023')
          

  data = pd.DataFrame({
      'Author': author,
      'Text': text,
      'Up-Down': up_down,
      'Year': year,
      'retrieved_utc': retrieved_utc,
      'created_utc': created_utc,
      'Retrieved_on': retrieved_on,
      'num_comments': num_comments
    })
  all_data = pd.concat([all_data, data], ignore_index=True)

all_data=all_data[all_data['Author']!='[deleted]']
all_data

r_ethtrader_posts_2016.jsonl
r_ethtrader_posts_2017.jsonl
r_ethtrader_posts_2018.jsonl
r_ethtrader_posts_2019.jsonl
r_ethtrader_posts_2020.jsonl
r_ethtrader_posts_2021.jsonl
r_ethtrader_posts_2022.jsonl
r_ethtrader_posts_2023.jsonl


,Author,Text,Up-Down,Year,retrieved_utc,created_utc,Retrieved_on,num_comments
0,carlslarson,How would a switch to Proof of Stake affect th...,10,2016,NaN,1427296704,1.440833e+09,9
1,heliumcraft,Price predictions?What do you think will be th...,12,2016,NaN,1427369350,1.440832e+09,27
2,StonedSheep,Exchange?Which exchanges will Ethereum first s...,5,2016,NaN,1427582991,1.440827e+09,7
4,heliumcraft,What software do you use for trading?,3,2016,NaN,1428015859,1.440817e+09,2
5,carlslarson,What factors will influence the price of Ether?,9,2016,NaN,1430401513,1.440763e+09,11
...,...,...,...,...,...,...,...,...
365557,MasterpieceLoud4931,"In 2023, the US government tried to kill crypto",7,2023,NaN,1704047821,1.704048e+09,1
365558,Prog132487,Biggest Crypto and NFT Games of 2023 - Decrypt,1,2023,NaN,1704051863,1.704052e+09,1
365559,teeceaustralia,Vitalik Buterin Reveals Ethereum’s Road Map Fo...,7,2023,NaN,1704059279,1.704059e+09,1
365560,aItalianStallion,"Chainlink News: CCIP, Data Feeds & Streams, Pr...",0,2023,NaN,1704064260,1.704064e+09,1


In [ ]:
def generalization(text):
  result = []
  for word in text.split(' '):
      if word.startswith('@') and len(word) > 1:
          word = '@user'
      elif word.startswith('http'):
          word = "http"
      elif word.startswith('www.'):
          word = "www"
      result.append(word)
  result = " ".join(result)
  return  result
all_data['Text'] = all_data['Text'].astype(str).apply(lambda x: generalization(x))
all_data

In [ ]:
def process_entry(x):
    return pd.Series({
        'sentiment': sentiment_analysis(x, model_sentiment, tokenizer_sentiment, labels_sentiment),
        'emotion': emotion_recognition(x, model_emotion, tokenizer_emotion, labels_emotion),
        'offensive': offensive_language(x, model_offensive, tokenizer_offensive, labels_offensive),
        'distilbert': distilbert(x),
        'ernie': ernie(x)
    })

batch_size = 500
start_time = time.time()

for start in range(0, len(genarlized_text['pre_processed_text']), batch_size):
    end = start + batch_size
    results_df = genarlized_text['pre_processed_text'][start:end].apply(process_entry)

    mode = 'w' if start == 0 else 'a'
    header = True if start == 0 else False

    results_df.to_csv('C:\\Users\\Castagna\\Desktop\\sentiment_emotion_offensive.csv', mode=mode, header=header, index=False)

    tempo_trascorso = time.time() - start_time
    print(f"Processed batch from {start} to {end}, Tempo trascorso: {tempo_trascorso / 60:.2f} minutes")

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer
model_reward = AutoModel.from_pretrained(
    "internlm/internlm2-7b-reward", 
    trust_remote_code=True,
)
tokenize_reward= AutoTokenizer.from_pretrained("internlm/internlm2-7b-reward", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
tokenizer_crypto_bert = AutoTokenizer.from_pretrained("ElKulako/cryptobert", use_fast=True)
model_crypto_bert= AutoModelForSequenceClassification.from_pretrained("ElKulako/cryptobert", num_labels = 3)
pipe = TextClassificationPipeline(model=model_crypto_bert, tokenizer=tokenizer_crypto_bert, max_length=64, truncation=True, padding = 'max_length')



In [ ]:

all_data['input_reward_model']=[[{"role": "user", "content": i}] for i in all_merged_first['Full Text'] ]

reward_model_df= pd.DataFrame()
crypto_bert_df= pd.DataFrame()
crypto_reward=pd.DataFrame()

def reward(x):
    return pd.Series({
        'reward_model': model_reward.get_score(tokenize_reward, x)
    })

def process_entry(x):
    return pd.Series({
        'crpto_bert': pipe(x)

    })

batch_size = 200
start_time = time.time()

for start in range(0, len(all_data['input_reward_model']), batch_size):
    end = start + batch_size
    reward_model_df = all_data['input_reward_model'][start:end].apply(reward)
    crypto_bert_df = all_data['Full Text'][start:end].apply(process_entry)
    unione = pd.concat([reward_model_df, crypto_bert_df], axis=0)

    crypto_reward = pd.concat([crypto_reward, unione], ignore_index=True)

    mode = 'w' if start == 0 else 'a'
    header = True if start == 0 else False
    crypto_reward.to_csv('C:\\Users\\Castagna\\Desktop\\reward_cryptoBert.csv', mode='a', header=False, index=False)

    elapsed_time = time.time() - start_time
    print(f"Processed batch from {start} to {end}, Tempo trascorso: {elapsed_time / 60:.2f} minutes")


In [ ]:
sentiment_dummies = pd.get_dummies(results_df['sentiment'], prefix='sentiment').astype(int)
emotion_dummies = pd.get_dummies(results_df['emotion'], prefix='emotion').astype(int)
offensive_dummies = pd.get_dummies(results_df['offensive'], prefix='offensive').astype(int)

encoded_emo_sent_off = pd.concat([
    sentiment_dummies,
    emotion_dummies,
    offensive_dummies
], axis=1)


encoded_emo_sent_off

In [ ]:
import pandas as pd
import textstat
import nltk
import re
import emoji
import spacy

nltk.download('punkt')
nlp = spacy.load("en_core_web_sm")

def calculate_readability_metrics(text):
    metrics = {}
    metrics['Lix'] = textstat.lix(text)
    metrics['Gunning Fog'] = textstat.gunning_fog(text)
    metrics['ARI'] = textstat.automated_readability_index(text)
    metrics['Coleman-Liau'] = textstat.coleman_liau_index(text)
    metrics['SMOG'] = textstat.smog_index(text)
    metrics['crypto_bert'] = pipe(text)
    
    return metrics

# Function to count emojis
def count_emojis(text):
    return len([char for char in text if char in emoji.EMOJI_DATA])

# Function to count links
def count_links(text):
    if isinstance(text, str):
        url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        return len(url_pattern.findall(text))
    return 0

# Function to calculate text statistics
def analyze_text(text):
    metrics = {}
    metrics['emoji_count'] = count_emojis(text)
    metrics['link_count'] = count_links(text)
    metrics['post_length'] = len(text)
    
    doc = nlp(text)
    metrics['word_count'] = len([token.text for token in doc if token.is_alpha])
    metrics['sentence_count'] = len(list(doc.sents))
    
    metrics['syllable_count'] = sum(count_syllables(token.text) for token in doc if token.is_alpha)
    metrics['avg_sentence_length'] = metrics['word_count'] / metrics['sentence_count'] if metrics['sentence_count'] > 0 else 0
    metrics['avg_syllables_per_word'] = metrics['syllable_count'] / metrics['word_count'] if metrics['word_count'] > 0 else 0
    
    # Flesch Reading Ease
    metrics['reading_ease'] = 206.835 - (1.015 * metrics['avg_sentence_length']) - (84.6 * metrics['avg_syllables_per_word'])
    # Flesch-Kincaid Grade Level
    metrics['grade_level'] = 0.39 * metrics['avg_sentence_length'] + 11.8 * metrics['avg_syllables_per_word'] - 15.59
    
    return metrics

# Combine the two analyses
def analyze_full_text(text):
    readability_metrics = calculate_readability_metrics(text)
    text_metrics = analyze_text(text)
    
    # Combine both dictionaries
    combined_metrics = {**readability_metrics, **text_metrics}
    return combined_metrics

combined_metrics = all_data['pre_processed_text'].apply(analyze_full_text)
final_metrics_df = pd.DataFrame(list(combined_metrics))
final_metrics_df


In [ ]:
Bearish=[]
Neutral=[]
Bullish=[]

for i in crypto_reward['crypto_bert']:
    if 'Neutral' in str(i):
        Neutral.append(1)
    else:
        Neutral.append(0)

    if 'Bullish' in str(i):
        Bullish.append(1)
    else:
        Bullish.append(0)

    if 'Bearish' in str(i):
        Bearish.append(1)
    else:
        Bearish.append(0)

final_metrics_df.drop('crypto_bert', axis=1, inplace=True)


final_metrics_df['Neutral']=Neutral
final_metrics_df['Bearish']=Bearish
final_metrics_df['Bullish']=Bullish
final_metrics_df

In [ ]:
lista=["Other label or No label","Adoption","Altcoin","Announcement","Comedy","Dapp","Discussion","Educational","Exchange","Fundamentals","Media","Metrics","Mining-Staking","News","Security","Sentiment","Strategy","Support","Technicals","Tool","Trading","Warning"]
lista_lower = [label.lower() for label in lista]

all_data['link_flair_text'] = all_data['link_flair_text'].str.lower()

all_data['link_flair_text'] = all_data['link_flair_text'].apply(
    lambda x: x if x in lista_lower else 'no label'
)
all_data

In [ ]:
link = pd.get_dummies(all_data['link_flair_text'], prefix='flair')
link = link.astype(int)

In [ ]:
all_data.reset_index(drop=True, inplace=True)
link.reset_index(drop=True, inplace=True)
final_metrics_df.reset_index(drop=True, inplace=True)
encoded_emo_sent_off.reset_index(drop=True, inplace=True)

total_data = pd.concat([
    all_data.drop(columns=['Author', 'Full Text', 'Year','link_flair_text', 'retrieved_utc', 'Retrieved_on'] ),
link,
final_metrics_df,
encoded_emo_sent_off
], axis=1)

total_data['created_utc'] = pd.to_datetime(total_data['created_utc'], unit='s')
total_data['Media']=total_data['Media'].notna().astype(int)
total_data['reward_model']=list(reward['reward_model'])
total_data

In [ ]:
def convert_to_tensor(input_str):
    cleaned_str = input_str.strip().replace("tensor(", "").replace(")", "").replace("[", "").replace("]", "")
    cleaned_str = cleaned_str.replace("\n", "").replace(" ", "")
    numbers = [float(num) for num in cleaned_str.split(",") if num]
    return torch.tensor(numbers)

def expand_vector_column(df, column_name, prefix):
    vectors = np.vstack(df[column_name])
    vector_df = pd.DataFrame(vectors, columns=[f"{prefix}_{i}" for i in range(vectors.shape[1])]) 
    return vector_df

total_data['distilbert'] =total_data['distilbert'].astype(str)
total_data['distilbert'] = total_data['distilbert'].apply(convert_to_tensor)
bert=expand_vector_column(total_data, 'distilbert', 'bert')    

total_data = pd.concat([total_data.drop(columns=['distilbert']), bert], axis=1)

In [ ]:
pre_total_data = all_data[all_data['created_utc'] < pd.to_datetime('2018-03-04')]
post_total_data = all_data[all_data['created_utc'] >= pd.to_datetime('2018-03-04')]

pre_total_data = pre_total_data.drop(columns=['created_utc'])
post_total_data = post_total_data.drop(columns=['created_utc'])

In [ ]:
pre_total_data.to_csv('c:\\Users\\aless\\Downloads\\data_pre_reward_x_final_classification.csv')
post_total_data.to_csv('c:\\Users\\aless\\Downloads\\data_post_reward_x_final_classification.csv')

# Final results

In [5]:
pre_total_data= pd.read_csv('insert path')      #'c:\\Users\\aless\\Downloads\\data_pre_reward_x_final_classification.csv'
pre_total_data

,Unnamed: 0,Up-Down,Num_Comments,Media,flair_adoption,flair_altcoin,flair_announcement,flair_comedy,flair_dapp,flair_discussion,...,bert_758,bert_759,bert_760,bert_761,bert_762,bert_763,bert_764,bert_765,bert_766,bert_767
0,0,10,9,0,0,0,0,0,0,0,...,-2.4693,-0.71732,-1.11810,1.05830,-1.3588,0.388220,-2.0808,-0.84124,-0.498630,-0.78077
1,1,12,27,0,0,0,0,0,0,0,...,-2.4375,-0.33521,-0.98417,0.60537,-1.3027,-0.005083,-1.9859,-0.75455,-0.063954,-0.88743
2,2,5,7,0,0,0,0,0,0,0,...,-2.7170,-0.36867,-0.92320,0.87915,-1.3446,-0.109510,-1.7503,-0.49715,-0.507740,-0.85654
3,3,3,2,0,0,0,0,0,0,0,...,-2.2256,-0.58304,-1.04297,0.90395,-1.0266,0.095971,-1.8184,-0.52708,-0.474220,-0.35750
4,4,9,11,0,0,0,0,0,0,0,...,-2.4307,-0.48110,-0.73013,0.77413,-1.0524,-0.086142,-1.8239,-0.80496,-0.171310,-0.37328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65690,1690,2,2,0,0,0,0,0,0,0,...,-2.6319,-0.45458,-0.37450,0.79077,-1.3192,0.251540,-1.4710,-0.75592,-0.331730,-0.46270
65691,1691,16,23,0,0,0,0,0,0,0,...,-2.6900,-0.56871,-0.51279,0.96698,-1.6035,0.041926,-1.6938,-0.54013,-0.327800,-1.17990
65692,1692,2,0,0,0,0,0,0,0,0,...,-2.5386,-0.85031,-0.28386,0.90991,-1.3002,0.056882,-1.7130,-0.39817,-0.341450,-0.82690
65693,1693,1,12,0,0,0,0,0,0,0,...,-2.8290,-0.67560,-0.40747,0.89618,-1.1387,0.051005,-1.8844,-0.41055,-0.140060,-0.75554


In [6]:
post_total_data= pd.read_csv('insert path')    #'c:\\Users\\aless\\Downloads\\data_post_reward_x_final_classification.csv
post_total_data

,Unnamed: 0,Up-Down,Num_Comments,Media,flair_adoption,flair_announcement,flair_comedy,flair_dapp,flair_discussion,flair_educational,...,bert_758,bert_759,bert_760,bert_761,bert_762,bert_763,bert_764,bert_765,bert_766,bert_767
0,0,2,6,0,0,0,0,0,0,0,...,-2.1102,0.69660,0.32780,0.148700,1.9013,-1.21450,0.083700,-0.70660,1.03970,0.70790
1,1,1,8,0,0,0,0,0,0,0,...,-2.0213,0.55942,0.14586,0.059950,1.9020,-0.88164,-0.091699,-0.55292,0.70170,1.12980
2,2,5,8,1,0,0,0,0,0,0,...,-2.0739,0.76920,0.66532,0.111300,2.0999,-1.16340,0.064000,-0.65240,0.97220,1.06120
3,3,124,21,1,0,0,0,0,0,0,...,-2.0176,0.94402,0.41523,0.256580,1.9387,-0.93738,-0.049934,-0.42100,0.89963,0.98649
4,4,20,4,0,0,0,1,0,0,0,...,-1.5878,0.68683,0.18753,0.059817,2.0965,-0.95004,0.421650,-0.78431,0.63633,0.89736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248315,13259,1,0,0,0,0,0,0,0,0,...,-1.9638,1.14360,2.58440,0.092437,2.0163,-0.90838,0.326670,-0.68376,0.51854,1.16360
248316,13260,1,0,0,0,0,0,0,0,0,...,-2.0271,0.77190,2.84336,0.286800,1.9576,-1.09590,0.138900,-0.68030,0.95610,1.00070
248317,13261,1,4,0,0,0,0,0,0,0,...,-2.0141,0.68621,3.12756,0.157220,1.7806,-0.99189,0.080964,-0.35149,0.71987,1.22440
248318,13262,1,0,0,0,0,0,0,0,0,...,-2.1588,0.89868,2.71220,0.039124,1.8470,-0.96300,0.001657,-0.54827,0.96191,1.03810
